In [1]:
from __future__ import print_function, division
import math

import numpy as np
import pandas as pd
import torch

In [ ]:

raw_data = pd.read_csv("C:\\Users\\coco1\\PycharmProjects\\PRHM01\\02\\data\\abalone.data", header=None, dtype=str)

In [6]:
amount = len(raw_data)
train_amount = int(amount * 0.7)
test_amount = amount - train_amount
train_set = raw_data.sample(frac=1).reset_index(drop=True).values

In [14]:
# define the LDF (Linear Discriminant Analysis)
# Under LDA we assume that the density for X, given every class k is following a Gaussian distribution. 
# LDA computes “discriminant scores” for each observation to classify what response variable class it is in 
# (i.e. default or not default). 
# These scores are obtained by finding linear combinations of the independent variables. 
# For a single predictor variable X = x

train_tensor = torch.tensor([[float(elem) for elem in x ] for x in train_set[:, 1:]], dtype=torch.float)
data_iter = train_tensor
label_iter = train_set[:,0]
        

In [15]:
train_data = torch.tensor(np.array(data_iter[0:train_amount]), dtype=torch.float)
test_data= torch.tensor(np.array(data_iter[train_amount:]), dtype=torch.float)
train_label = label_iter[0:train_amount]
test_label = label_iter[train_amount:]

In [28]:
type1_train_data = []
type2_train_data = []
type3_train_data = []
for i in range(train_label.shape[0]) :
    if str(train_label[i]) == "M" : type1_train_data.append(train_data[i])
    if str(train_label[i]) == "F" : type2_train_data.append(train_data[i])
    if str(train_label[i]) == "I" : type3_train_data.append(train_data[i])
    # print(str(train_label[i]))

In [37]:
# get the mu
def get_qda_norm_attribute (train, all_size) : 
    init_vec = torch.zeros(train[0].size()[0], dtype=torch.float)
    for vec in train :
        init_vec += vec
    return init_vec / len(train), len(train) / all_size
def get_qda_cor_matrix (train, mu) :
    print(len(train))
    size = train[0].shape[0]
    zeros = torch.zeros((size,size))
    for vec in train : 
        # print((vec - mu).size())
        zeros += (vec - mu).view(-1, 1).mm((vec - mu).view(1,-1))
    return zeros / len(train)
def get_qda_cor_inv (train, mu) :
    return np.linalg.inv(get_qda_cor_matrix(train, mu))

In [38]:
#combining together
all_size = train_data.size()[0]
mu_1, pi_1 = get_qda_norm_attribute(type1_train_data, all_size)
mu_2, pi_2 = get_qda_norm_attribute(type2_train_data, all_size)
mu_3, pi_3 = get_qda_norm_attribute(type3_train_data, all_size)
cor_ma_1 = get_qda_cor_matrix(type1_train_data, mu_1)
cor_ma_2 = get_qda_cor_matrix(type2_train_data, mu_2)
cor_ma_3 = get_qda_cor_matrix(type3_train_data, mu_3)

cor_ma_inv_1 = get_qda_cor_inv(type1_train_data, mu_1)
cor_ma_inv_2 = get_qda_cor_inv(type2_train_data, mu_2)
cor_ma_inv_3 = get_qda_cor_inv(type3_train_data, mu_3)

1068
930
925
1068
930
925


In [44]:
def qdf(x, mu, pi, cor_inv) :
    return -1 * (x.view(1,-1).mm(cor_inv)).mm(x.view(-1,1))  +\
           2 * (mu.view(1,-1).mm(cor_inv)).mm(x.view(-1,1)) - \
           (mu.view(1,-1).mm(cor_inv)).mm(mu.view(-1,1)) + \
           2 * math.log2(pi)
right = 0
result_dic = dict()
result_count_dict = dict()
for i in range(test_data.size()[0]) :
    vec = test_data[i]
    label = test_label[i]
    result = 0
    p1 = qdf(vec, mu_1, pi_1, cor_ma_1)
    p2 = qdf(vec, mu_2, pi_2, cor_ma_2)
    p3 = qdf(vec, mu_3, pi_3, cor_ma_3)
    if p1 > p2 :
        if p1 > p3 : result = "M"
        else : result = "I"
    else :
        if p2 > p3 : result = "F"
        else : result = "I"
    if result == label :
        if result_dic.__contains__(label) : 
            result_dic[label] += 1
        else:
            result_dic[label] = 1
        right += 1
print(right / (32 - 20))
for label in test_label :
    if result_count_dict.__contains__(label) : 
        result_count_dict[label] += 1
    else:
        result_count_dict[label] = 1
for l in ["M","F", "I"] :
    print(l ,"类准确度 ：" , result_dic[l] / result_count_dict[l])
# 50.75

50.75
M 类准确度 ： 0.20652173913043478
F 类准确度 ： 0.46153846153846156
I 类准确度 ： 0.815347721822542
